Final Project job skills database data preprocessing, prepartion and target tables loading

0. Data Preprocessing, make sure all text file can be read in, move cannot read ones to job_ErrorHold folder

In [34]:

import os
rootpath = "C:\DS_FinalProject_Data\Jobs\\"

# if there is file in job upoad folder, clear the SQLite Jobs table for reload
num_jobs = 0 # number of sample jobs per job board

# build job file list, clear Sqlite Jobs table for reload
rootdir = os.listdir(rootpath)
if rootdir != []:
    for afile in rootdir:
    # check if current path is a file
        ffpath = os.path.join(rootpath, afile)
        if os.path.isfile(ffpath):
            with open(ffpath,"r") as jobfile:
                try:
                    joblines = jobfile.readlines()
                except:
                    
                    tfpath = ffpath.replace('Jobs','Jobs_ErrorHold')
                    move_stmt = 'move '+ffpath+' '+tfpath
                    os_cmd = 'cmd /k "'+move_stmt+'"'
#                     print(os_cmd)                   
                    os.system(os_cmd)
                    print(ffpath)

1. Data Preparation for Skills table - load skills text file and build SQLlite Skills master table

In [35]:
import os
import pandas as pd
from pandas import Series, DataFrame

# Connect to SQLite3 database
import sqlite3
conn = sqlite3.connect('C:\\DS_FinalProject_Data\\JobSkill.db')
curs = conn.cursor()

# read job skill text file, build and insert the skill list to SQLite Skills table 
# for each tech, nontech, software, platform type

f_path = "C:\DS_FinalProject_Data\Skills\Skills.csv"

if os.path.isfile(f_path):
        with open(f_path,"r") as f_skills:
            skills = pd.read_csv(f_path)
skmlst = Series(skills.iloc[:,0])
sktypes = Series(skills.iloc[:,1])
skalts = Series(skills.iloc[:,2])
num_sk = len(skmlst)
# drop the existing skills in Skills table
if num_sk:
    del_stmt = 'DELETE FROM skills WHERE Skill_ID IN (SELECT skill_ID FROM skills);'
    curs.execute(del_stmt)
    
# reset primary key autoincrement to 1
skreset_stmt = '''UPDATE 'sqlite_sequence' SET seq = 0 WHERE name = 'Skills';'''
curs.execute(skreset_stmt)

# insert new skills from list to Skills table
sktype = 'TECH'
ins_stmt = 'INSERT INTO skills (Description, Type, JobCount, AlternateSkill) VALUES(?,?,?,?)'
for i in range(num_sk):
    skdesc = str(skmlst[i]).lower()
    sktype = str(sktypes[i]).lower()
    skalt = str(skalts[i]).lower()
    curs.execute(ins_stmt,(skdesc,sktype,0,skalt))
    conn.commit()


2. Data Preparation for Jobs table - load jobs text files and build SQLlite Jobs table

In [36]:
import os

# build job board dictionary 
jboards = {"G":"Glassdoor","I":"Indeed","L":"LinkedIn","M":"Monster"}
jbkey_list = list(jboards.keys())
jbval_list = list(jboards.values())

import sqlite3
conn = sqlite3.connect('C:\\DS_FinalProject_Data\\JobSkill.db')
curs = conn.cursor()
# build hard skill ID, description master dictionaries
selhsk_stmt = '''SELECT Skill_ID, Description FROM Skills WHERE TYPE IN ('cs','ds','pltfm','stat','tech') ORDER BY DESCRIPTION;'''
skhard = {}
for skkey, skval in curs.execute(selhsk_stmt):
    skhard[skkey] = skval

skhardkeys = list(skhard.keys())
skhardvals = list(skhard.values())

# build soft skill ID, description master dictionaries
selssk_stmt = '''SELECT Skill_ID, Description FROM Skills WHERE TYPE IN ('ba','domn','nontech') ORDER BY DESCRIPTION;'''
sksoft = {}
for skkey, skval in curs.execute(selssk_stmt):
    sksoft[skkey] = skval

sksoftkeys = list(sksoft.keys())
sksoftvals = list(sksoft.values())
num_hsk = len(skhardkeys)
num_ssk = len(sksoftkeys)

# all skill dictionaries
skmall = skhard
skmall.update(sksoft)

# set sqlite statements
insjob_stmt = '''INSERT INTO Jobs(DatePosted, JobTitle, Company, Location, Industry, HardSkill, SoftSkill,AllSkill,\
                                HardSkillDesc, SoftSkillDesc, AllSkillDesc, URL, Source)\
                                VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?);'''
# read raw job text file
rootpath = "C:\DS_FinalProject_Data\Jobs\\"

# if there is file in job upoad folder, clear the SQLite Jobs table for reload
# num_jobs = 0 # number of sample jobs per job board

# build job file list, clear Sqlite Jobs table for reload
rootdir = os.listdir(rootpath)
flist = []
if rootdir != []:
    for afile in rootdir:
    # check if current path is a file
        ffpath = os.path.join(rootpath, afile)
        if os.path.isfile(ffpath):
            flist.append(ffpath)
            
    deljob_stmt = 'DELETE FROM Jobs WHERE Job_ID IN (SELECT Job_ID FROM Jobs);'
    curs.execute(deljob_stmt)
    
    # reset primary key autoincrement to 1
    jreset_stmt = '''UPDATE 'sqlite_sequence' SET seq = 0 WHERE name = 'Jobs';'''
    curs.execute(jreset_stmt)
    
    conn.commit()


2 Data Preparation & load Jobs table continue ...
  loop through each job files from job boards in the stored directory

In [37]:
       
def parseskills(body):  
#     print(body)
    skhard = []
    skharddesc = []
    sksoft = []
    sksoftdesc = []
    for line in body:         # loop through each job line
        line = str(line).lower()
        for i in range(num_hsk):    # loop through each of the hard skill list
            skill = skhardvals[i] 
            if skill == 'r': skill = ' r, '
            if skill in line:
                if skill == ' r, ': skill = 'r' # reverse from ' r, ' to 'r'
                skharddesc.append(skill)
                skhard.append(skhardkeys[i])
        for i in range(num_ssk):    # loop through each of the soft skill list
            skill = sksoftvals[i]             
            if skill in line:
                sksoftdesc.append(skill)
                sksoft.append(sksoftkeys[i])
    skharddescu = set(skharddesc)          # use set to make sure the list only containing unique skills 
    skhardu = set(skhard)
    skharddesc = list(skharddescu)
    skhard = list(skhardu)
    sksoftdescu = set(sksoftdesc)          # use set to make sure the list only containing unique skills 
    sksoftu = set(sksoft)
    sksoftdesc = list(sksoftdescu)
    sksoft = list(sksoftu)   
    skharddesc.sort()
    skhard.sort()
    sksoftdesc.sort()
    sksoft.sort()
    return skhard, skharddesc, sksoft, sksoftdesc
#
# main process start
#
for ffpath in flist:
    fprefix_pos = ffpath.rfind('\\')
    fprefix = (ffpath[fprefix_pos+1:fprefix_pos+2]).upper()
    if fprefix in (jbkey_list) and os.path.isfile(ffpath):
        dtposted = ''; title = ''; company = ''; location = ''; industry = ''; sktech = ''
        sw = ''; pltfm = ''; sknontech = ''; url = ''; source = ''
        with open(ffpath,"r") as jobfile:
            joblines = jobfile.readlines()
            source = jboards[fprefix]
        
# extract job title, date posted, city, state, company, source
             
            line1 = joblines[0]
            line2 = joblines[1]
            line3 = joblines[2]
            line4 = joblines[3]
            line5 = joblines[4]
            if fprefix == "G":
                company = line1.strip()
                title = line3.strip()
                location = line4.strip()
                dtposted = '11/20/22'
                body = joblines[4:]
                skhard, skharddesc, sksoft, sksoftdesc = parseskills(body)
            elif fprefix == "I":
                url = line1.strip()
                # title line1 before '-'
                end = line2.find("-")
                title = line2[:end].strip() # job title                
                dtposted = '11/16/2022' # date posted
                company = line3.strip()
                location = line5.strip()
                body = joblines[5:]
                skhard, skharddesc, sksoft, sksoftdesc = parseskills(body)
            elif fprefix == "L":                
# line1
                beg = line1.find(":")+1
                url = line1[beg:].strip() # url

# line2
                end = line3.find(",")
                if end <= 0:
                    end = line3.find("(")
                title = line3[:end].strip() # job tilte           
                     
                beg =line3.find("Time Stamp")
                if beg <= 0:
                    beg=line3.find("Date Posted")-1
                beg = beg + 13 
                end = beg + 10
                dtposted = line3[beg:end].strip() # date posted
                x = dtposted.find(",")
            
                if x > 0:
                    dtposted = dtposted[:x]
                                
                x = title.find("(Time Stamp")
                if x > 0:
                    title = title[:x].strip() # if not having ',' in line1
# line3
                beg = line4.find("  ")
                company = line4[:beg].strip() # company
                location = line4[beg:].strip() # location
# extract job skills into a list 
# line 3 - end     
                body = joblines[4:]
                skhard, skharddesc, sksoft, sksoftdesc = parseskills(body)  
            elif fprefix == "M":
                title = line1.strip()
                company = line2.strip()
                location = line3.strip()
                dtposted = '11/20/22'
                body = joblines[4:]
                skhard, skharddesc, sksoft, sksoftdesc = parseskills(body)

            odata = (str(company)+str(location)+str(sktech)+str(sw)+str(pltfm)+str(sknontech).strip())    
            if len(odata) > 0:
                skall = skhard+sksoft
                skalldesc = skharddesc+sksoftdesc
                skall.sort()
                skalldesc.sort()
                skhard = ",".join(str(e) for e in skhard)
                skharddesc = ",".join(str(e) for e in skharddesc)
                sksoft = ",".join(str(e) for e in sksoft)
                sksoftdesc = ",".join(str(e) for e in sksoftdesc)
                skall = ",".join(str(e) for e in skall)
                skalldesc = ",".join(str(e) for e in skalldesc)
                                
                curs.execute(insjob_stmt,(dtposted,title,company,location,industry,\
                                          str(skhard),str(sksoft),str(skall),str(skharddesc),str(sksoftdesc),str(skalldesc),\
                                          url,source))

                conn.commit()
             

3. Data Preparation - update skill job count column

In [38]:

sjcsel_stmt = '''SELECT JobCount FROM Skills WHERE Skill_ID = ?;'''
sjcupd_stmt = '''UPDATE Skills SET JobCount = ? WHERE Skill_ID = ?;'''

jsel_stmt = '''SELECT Job_ID, AllSkill FROM Jobs;'''
curs.execute(jsel_stmt)
jobtechs = curs.fetchall()

conn.commit()
# print(jobtechs)
for jobtech in jobtechs:
    jobID = jobtech[0]
    tkskills = jobtech[1]
#     print(tkskills)    
    
    for skill_ID in tkskills.split(','):
#         print(skill_ID)
        curs.execute(sjcsel_stmt,(skill_ID,))
        jcnts= curs.fetchall()
#         print(type(jcnt),' jcnt=',jcnt)
        for jcnt in jcnts:
            jcnt = int(jcnt[0]) + 1

        curs.execute(sjcupd_stmt,(jcnt,skill_ID))
        conn.commit()
#         print('skillid=',skill_ID,' jcnt=',jcnt)
        

4. Data Preparation and load SkillsJobs cross reference table 

In [39]:
# clear SkillsJobs table

delskjob_stmt = 'DELETE FROM SkillsJobs WHERE SkillsJobs_ID IN (SELECT SkillsJobs_ID FROM SkillsJobs);'
curs.execute(delskjob_stmt)
    
# reset primary key autoincrement to 1
skjreset_stmt = '''UPDATE 'sqlite_sequence' SET seq = 0 WHERE name = 'SkillsJobs';'''
curs.execute(skjreset_stmt)
    
conn.commit()



4. Data Preparation and load SkillsJobs cross reference table - continue ...

In [40]:
sjsel_stmt = '''SELECT SkillsJobs_ID, JobIDs, RoleIDs FROM SkillsJobs WHERE SkillIDs = ?;'''
sjupd_stmt = '''UPDATE SkillsJobs SET JobIDs = ?, RoleIDs = ?, JobCnt = ?, RoleCnt = ? WHERE SkillsJobs_ID = ?;'''
sjins_stmt = '''INSERT INTO SkillsJobs (SkillIDs, JobIDs, RoleIDs, JobCnt, RoleCnt) VALUES(?,?,?,?,?);'''

jsel_stmt = '''SELECT Job_ID, AllSkill, JobTitle FROM Jobs;'''
curs.execute(jsel_stmt)
jobtechs = curs.fetchall()

conn.commit()

for jobtech in jobtechs:
    jobID = jobtech[0]
    tkskills = jobtech[1]
    jobtitle = jobtech[2]
    #print(jobtitle)
    
    curs.execute(sjsel_stmt,([tkskills]))
    sjrec = curs.fetchall()
    #print(sjrec)
    if sjrec == []:
        curs.execute(sjins_stmt,(tkskills, jobID, jobtitle, 1, 1))
    else:
        sjrec = list(sjrec[0])
        #print(sjrec[0],sjrec[1],sjrec[2])
        skilljobID = sjrec[0]
        jobIDs = sjrec[1]
        jobtitles = sjrec[2]

        jobIDs = jobIDs + ','+ str(jobID)
        jobtitles = jobtitles + ',' + str(jobtitle)
        
        jobIDsu = str(set(jobIDs))              # using set to make sure unique values
        jobtitlesu = str(set(jobtitles))        # using set to make sure unique values
        
        jobcnt = len(jobIDsu.split(','))
#         print(jobcnt)
        titlecnt = len(jobtitlesu.split(','))
#         print(titlecnt)
        curs.execute(sjupd_stmt,(jobIDs, jobtitles, jobcnt, titlecnt, skilljobID))
        
    conn.commit()  

5 Data Preparation and load JobSkillParis table

In [41]:
# clear JobSkillPairs table

deljsp_stmt = 'DELETE FROM JobSkillPairs WHERE JSP_ID IN (SELECT JSP_ID FROM JobSkillPairs);'
curs.execute(deljsp_stmt)
    
# reset primary key autoincrement to 1
jspset_stmt = '''UPDATE 'sqlite_sequence' SET seq = 0 WHERE name = 'JobSkillPairs';'''
curs.execute(jspset_stmt)
    
conn.commit()


5 Data Preparation and load JobSkillPairs table continue ...

In [ ]:
# build skill pair support% confidence%
# skill A is required, the probility of C skill also required is __
# A is highly associate with C. If A is required, it is likely that C will also be required
# inspair_stmt = '''INSERT INTO JobSkillPairs (skID1, skID2, skDesc1, skDesc2,skJcnt1, skJcnt2, UnionJCnt, TotJcnt) \
#                                 VALUES(?,?,?,?,?,?,?,?);'''
inspair_stmt = '''INSERT INTO JobSkillPairs (skID1,skID2,skDesc1,skDesc2,\
                        skJcnt1,skJcnt2,UnionJcnt,TotJcnt,support1,support2,\
                        confidence,lift) \
                        VALUES(?,?,?,?,?,?,?,?,?,?,?,?);'''
selpair_stmt = '''SELECT JSP_ID, UnionJcnt FROM JobSkillPairs WHERE skID1 = ? \
                                AND skID2 = ?;'''
selpair2_stmt = '''SELECT JSP_ID, skJCnt1, skJCnt2, UnionJcnt, TotJCnt FROM JobSkillPairs;'''
updtpair1_stmt = '''UPDATE JobSkillPairs SET UnionJcnt = ? WHERE JSP_ID = ?'''
updtpair2_stmt = '''UPDATE JobSkillPairs SET support1 = ?, support2 = ?,\
                                confidence = ?, lift = ? \
                                WHERE JSP_ID = ?'''

selsk_stmt = '''SELECT AllSkill, AllSkillDesc FROM Jobs;''' 

selskcnt_stmt = '''SELECT Skill_ID, JobCount FROM Skills;'''

import sqlite3
conn = sqlite3.connect('C:\\DS_FinalProject_Data\\JobSkill.db')
curs = conn.cursor()
# build skill and job count master list
curs.execute(selskcnt_stmt)
skall = curs.fetchall()
skmst = {}

for skillid, jobcount in curs.execute(selskcnt_stmt):
    skmst[skillid] = jobcount
# loop 1: select job skills from all jobs for processing  
curs.execute(selsk_stmt)
jsksall = curs.fetchall()
totjcnt = len(jsksall)
# get job skills from the next job
for jskone, jskdescone in jsksall: 
    numsk = len(jskone.split(',')) # number of skills required by the job
    if numsk >= 2:                 # if greater than 2 build the pair(s)
        jskone = jskone.split(',') # convert string to list to use pop()
        jskdescone = jskdescone.split(',') # convert string to list to use pop()
        skid1 = jskone.pop(0)      # get 1st skill from job skill list for the job
        skdesc1 = skmall.get(int(skid1))        
        for item in jskone:        # pair the popped skill with the rest of job skills in the job list
            skid2 = item
            skdesc2 = skmall.get(int(skid2))
            curs.execute(selpair_stmt,(skid1,skid2))
            res = curs.fetchall()               
            if res == []:
                jcnt1 = skmst[int(skid1)]
                jcnt2 = skmst[int(skid2)]
                unioncnt = 1
                curs.execute(inspair_stmt,(skid1,skid2,skdesc1,skdesc2,jcnt1,jcnt2,unioncnt,totjcnt,'0','0','0','0'))
                conn.commit()
            else:
                for JSP_ID, unioncnt in res:
                    unioncnt += 1                        
                    curs.execute(updtpair1_stmt,(unioncnt,JSP_ID))
                    conn.commit()
            # repeat for skid2, skid1 (reverse of skid1, skid2)
            curs.execute(selpair_stmt,(skid2,skid1))
            res = curs.fetchall()  
            if res == []:
                jcnt1 = skmst[int(skid1)]
                jcnt2 = skmst[int(skid2)]
                unioncnt = 1
                curs.execute(inspair_stmt,(skid2,skid1,skdesc2,skdesc1,jcnt2,jcnt1,unioncnt,totjcnt,'0','0','0','0'))
                conn.commit()
            else:
                for JSP_ID, unioncnt in res:
                    unioncnt += 1                        
                    curs.execute(updtpair1_stmt,(unioncnt,JSP_ID))
                    conn.commit()
                    
            
# Calculate support, confidence, lift for all jobskillpairs records
curs.execute(selpair2_stmt)
res = curs.fetchall() 
for JSP_ID, skjcnt1, skjcnt2, unionjcnt, totjcnt in res:
    support1 = skjcnt1 / totjcnt    
    support2 = skjcnt2 / totjcnt
    supportunion = unionjcnt / totjcnt
    confidence = supportunion / support1 * 100
    lift = confidence / support2
    curs.execute(updtpair2_stmt,(round(support1,2), round(support2,2), round(confidence,0), round(lift,0), JSP_ID))
    conn.commit()   

# interpreting resul
# select skDesc1, skDesc2, confidence, lift '> 1 highly associate, < 1 unlikely associate, = 1 not associate' from JobSkillPairs order by confidence desc
                    
